In [ ]:
# -*- coding: utf-8 -*-

# Install required packages
%pip install pandas pillow numpy matplotlib ipywidgets


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


NameError: name 'pd' is not defined

In [6]:
import pandas as pd

# create a dataframe with sample data
df = pd.DataFrame(
    {
        "Name": ["Alice", "Bob", "Charlie"],
        "Age": [25, 30, 35],
        "Gender": ["F", "M", "M"],
    }
)
print(df)

      Name  Age Gender
0    Alice   25      F
1      Bob   30      M
2  Charlie   35      M


In [ ]:

# Accept image path and optional alias from user input

while True:
    image_path = input("Enter the image path (folder or zip file only): ")
    if image_path.lower().endswith(('.zip', '/')):
        break
    print("Invalid input. Please enter a folder path or a zip file.")
alias = input("Enter an optional alias (press Enter to skip): ")

print(f"Image path: {image_path}")
if alias:
    print(f"Alias: {alias}")
else:
    print("No alias provided.")

Image path: /run/media/luxcium/2TB-Seagate/MJ-backups/others/generation_361fee5e-9471-4c5b-ab1c-8f5fcd216b47_images.zip
Alias: mj-others-test


In [ ]:
import os
from zipfile import ZipFile
from PIL import Image
import ipywidgets as widgets
from IPython.display import display, clear_output
import numpy as np
import matplotlib.pyplot as plt

# Function to extract images from a zip file or list images in a folder
def get_image_paths(path):
    image_paths = []
    if path.endswith('.zip'):
        with ZipFile(path, 'r') as zip_ref:
            # Extract to a temporary folder
            temp_dir = 'temp_images'
            if not os.path.exists(temp_dir):
                os.makedirs(temp_dir)
            zip_ref.extractall(temp_dir)
            image_paths = [os.path.join(temp_dir, file) for file in zip_ref.namelist() 
                         if file.lower().endswith(('png', 'jpg', 'jpeg', 'gif', 'bmp')) 
                         and not os.path.basename(file).startswith('.')]
    else:
        image_paths = [os.path.join(path, file) for file in os.listdir(path) 
                     if file.lower().endswith(('png', 'jpg', 'jpeg', 'gif', 'bmp')) 
                     and not file.startswith('.')]
    
    return sorted(image_paths)

# Class for batch processing and visualizing images
class ImageBatchProcessor:
    def __init__(self, image_paths, batch_size=9):
        self.image_paths = image_paths
        self.batch_size = batch_size
        self.current_batch = 0
        self.total_batches = max(1, (len(image_paths) + batch_size - 1) // batch_size)
        self.setup_widgets()
    
    def setup_widgets(self):
        self.output = widgets.Output()
        self.prev_button = widgets.Button(description="Previous Batch")
        self.next_button = widgets.Button(description="Next Batch")
        self.batch_slider = widgets.IntSlider(
            value=1,
            min=1,
            max=max(1, self.total_batches),
            step=1,
            description='Batch:',
            continuous_update=False
        )
        self.batch_size_slider = widgets.IntSlider(
            value=self.batch_size,
            min=1,
            max=25,
            step=1,
            description='Batch Size:',
            continuous_update=False
        )
        
        # Connect callbacks
        self.prev_button.on_click(self.on_prev_click)
        self.next_button.on_click(self.on_next_click)
        self.batch_slider.observe(self.on_batch_change, names='value')
        self.batch_size_slider.observe(self.on_batch_size_change, names='value')
        
        # Create UI layout
        self.nav_buttons = widgets.HBox([self.prev_button, self.next_button])
        self.sliders = widgets.VBox([self.batch_slider, self.batch_size_slider])
        self.controls = widgets.HBox([self.nav_buttons, self.sliders])
        self.main_ui = widgets.VBox([self.controls, self.output])
    
    def on_prev_click(self, _):
        if self.current_batch > 0:
            self.current_batch -= 1
            self.batch_slider.value = self.current_batch + 1
            self.display_batch()
    
    def on_next_click(self, _):
        if self.current_batch < self.total_batches - 1:
            self.current_batch += 1
            self.batch_slider.value = self.current_batch + 1
            self.display_batch()
    
    def on_batch_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            self.current_batch = change['new'] - 1
            self.display_batch()
    
    def on_batch_size_change(self, change):
        if change['type'] == 'change' and change['name'] == 'value':
            self.batch_size = change['new']
            self.total_batches = max(1, (len(self.image_paths) + self.batch_size - 1) // self.batch_size)
            self.batch_slider.max = max(1, self.total_batches)
            if self.current_batch >= self.total_batches:
                self.current_batch = self.total_batches - 1
                self.batch_slider.value = self.current_batch + 1
            self.display_batch()
    
    def display_batch(self):
        with self.output:
            clear_output(wait=True)
            
            start_idx = self.current_batch * self.batch_size
            end_idx = min(start_idx + self.batch_size, len(self.image_paths))
            batch_paths = self.image_paths[start_idx:end_idx]
            
            # Calculate grid dimensions based on batch size
            grid_size = int(np.ceil(np.sqrt(len(batch_paths))))
            fig, axes = plt.subplots(grid_size, grid_size, figsize=(15, 15))
            
            # Ensure axes is a 2D array even for a single image
            if grid_size == 1:
                axes = np.array([[axes]])
            elif len(axes.shape) == 1:
                axes = axes.reshape(1, -1)
            
            # Display images in the current batch
            for i, img_path in enumerate(batch_paths):
                row, col = divmod(i, grid_size)
                try:
                    img = Image.open(img_path)
                    axes[row, col].imshow(img)
                    axes[row, col].set_title(f"{os.path.basename(img_path)}")
                    axes[row, col].axis('off')
                except Exception as e:
                    axes[row, col].text(0.5, 0.5, f"Error: {str(e)}", 
                                       ha='center', va='center', color='red')
            
            # Hide unused axes
            for i in range(len(batch_paths), grid_size * grid_size):
                row, col = divmod(i, grid_size)
                axes[row, col].axis('off')
            
            plt.tight_layout()
            plt.subplots_adjust(hspace=0.3)
            plt.show()
            
            print(f"Showing batch {self.current_batch + 1}/{self.total_batches} " +
                  f"(Images {start_idx + 1}-{end_idx} of {len(self.image_paths)})")
    
    def display(self):
        display(self.main_ui)
        self.display_batch()

# Create a function to initialize the image batch processor with a path
def process_images(path):
    paths = get_image_paths(path)
    if paths:
        print(f"Found {len(paths)} images. Setting up batch processor...")
        processor = ImageBatchProcessor(paths)
        processor.display()
    else:
        print("No images found in the provided path.")

# Create a path input widget
path_input = widgets.Text(
    value='',
    placeholder='Enter path to image folder or zip file',
    description='Image Path:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

# Create a button to process images
process_button = widgets.Button(
    description='Process Images',
    button_style='primary',
    layout=widgets.Layout(width='auto')
)

# Output widget for displaying messages
output = widgets.Output()

# Define the button click handler
def on_process_button_clicked(_):
    with output:
        clear_output()
        if path_input.value:
            print(f"Processing images from: {path_input.value}")
            process_images(path_input.value)
        else:
            print("Please enter a valid path to an image folder or zip file.")

# Connect the button click handler
process_button.on_click(on_process_button_clicked)

# Display the UI
display(widgets.VBox([
    widgets.HTML("<h3>Image Batch Processor</h3>"),
    widgets.HBox([path_input, process_button]),
    output
]))